# Data exploration

### Contents
1. imports
1. [load subset](#subsetLoad)

In [1]:
import laspy
from scipy.spatial import cKDTree
import numpy as np
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from sklearn.preprocessing import MinMaxScaler


import geopandas as gpd
from shapely.geometry import Polygon, Point
import pandas as pd
pd.set_option('display.max_columns', 100)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
# preprocessing and scoring
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestClassifier



In [ ]:
def get_spatial_subset(
    full_cloud, xmin: int, xmax: int, ymin: int, ymax: int
) -> laspy.lasdata.LasData:
    """
    Create a subset of a LAS file based on a bounding box in coordinates of the cloud file

    parameters:
    full_cloud: a laspy cloud object you want to subset from

    xmin: the values of the bounding box object
    xmax
    ymin
    ymax

    returns: laspy.lasdata.LasData object
    """

    # create empty laspy collection to put the filtered points in that has the same format and file version as the original
    new_file = laspy.create(
        point_format=cloud.header.point_format, file_version=cloud.header.version,
    )

    # create matrices of boolean values
    a = cloud.x > xmin
    b = cloud.x < xmax
    c = cloud.y > ymin
    d = cloud.y < ymax
    # subset the points and put them into the new laspy file
    new_file.points = full_cloud.points[a & b & c & d]

    new_file.header.offsets = full_cloud.header.offsets
    new_file.header.scales = full_cloud.header.scales

    return new_file


In [ ]:
# load in the fully LAZ file
cloud = laspy.read(r"./data/AHN4_Noordwijk.laz")


In [ ]:
cloud.X

In [ ]:
# define the area of interest
xcenter = 89401.42414516   #89400
bbbox_size_length= 170           #300
bbbox_size_width = 100           #300
ycenter = 472877.83174913  #472868
xmin = xcenter - 0.5 * bbbox_size_length
xmax = xcenter + 0.5 * bbbox_size_length
ymin = ycenter - 0.5 * bbbox_size_width
ymax = ycenter + 0.5 * bbbox_size_width

# create the subset
subset = get_spatial_subset(cloud, xmin, xmax, ymin, ymax)


# saving the bounds as a shapefile

lat_point_list = [ymin, ymin, ymax, ymax, ymin] 
lon_point_list = [xmin, xmax ,xmax, xmin, xmin]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = gpd.GeoDataFrame(index=[0], epsg=28992, geometry=[polygon_geom])       
print(polygon.geometry)

# polygon.to_file(filename='./data/bounds_new.geojson', driver='GeoJSON') # geojson export has a problem with how it is saved, doesnt render in QGIS
polygon.to_file(filename='./data/bounds_new.shp', driver="ESRI Shapefile")


In [ ]:
# create a tree for all points in the subset
dataset = np.vstack((subset.X, subset.Y, subset.Z)).transpose()
tree = cKDTree(dataset)

In [ ]:
# setting up the extra dimensions to add back to the las file
dimension = laspy.point.format.ExtraBytesParams("normal_angle", "float64")
subset.add_extra_dim(dimension)
# check that it was added correctly
list(subset.point_format.extra_dimension_names)


In [ ]:
# get an array for all the points
all_points = subset.points.array
all_points


In [ ]:
def _fit_plane_pca(point, tree, k=8) -> PCA:
    """
    internal function designed to fit a PCA model to the k nearest points to a point
    parameters:
    point: single point as structured ndarray of point data returned from laspy.points.array()
    tree: the KD tree for the points
    k: number of neighbors to consider

    returns:
    fitted pca model: sklean.decom
    """
    distances, neighbors_indices = tree.query((point[0], point[1], point[2]), k=8)

    neighbors_points = tree.data[neighbors_indices]
    # set up pca model
    pca = PCA(n_components=3)
    # fit the model to the points
    pca.fit(neighbors_points)
    return pca

def _find_angle(vector,axis='z'):
    axis_lookup = {'z':[0,0,1],'y':[0,1,0],'x':[1,0,0]}
    unit_vector = axis_lookup[axis]
    # find the angle with the z axis using the dot product
    angle = np.rad2deg(np.arccos(vector.dot(unit_vector)))

    # we want all the normals pointing in the same direction

    if angle > 90:
        # if the vector is pointing the wrong way, flip it
        vector = vector * -1
        # find the angle of the new vector
        angle = np.rad2deg(np.arccos(vector.dot(unit_vector)))

    return angle



In [ ]:
def pca_3d_shape(point_array: np.array, tree: cKDTree, k: int = 8) -> np.array:
    """
    takes an array of points from laspy, a kd tree, and an integer number of neighbors to sample


    returns an array of:

    linearity, planarity, scattering, omnivariance, anisotropy, eigenentropy, eigsum, ch_cur

    """
    length_parray = len(point_array)
    normal_angle = np.zeros(length_parray)
    normal_vectors = np.zeros((length_parray, 3))
    # set up empty arrays
    linearity = np.zeros(length_parray)
    planarity = np.zeros(length_parray)
    scattering = np.zeros(length_parray)
    omnivariance = np.zeros(length_parray)
    anisotropy = np.zeros(length_parray)
    eigenentropy = np.zeros(length_parray)
    eigsum = np.zeros(length_parray)
    ch_cur = np.zeros(length_parray)
    norm_ang_x = np.zeros(length_parray)
    norm_ang_y = np.zeros(length_parray)

    for i, point in enumerate(point_array):

        # get a fitted pca model
        pca = _fit_plane_pca(point, tree, k=8)

        # get the Z value for the normal
        normal_vector = pca.components_.T[2]

        # find the angle with the z axis using the dot product
        angle = _find_angle(normal_vector,'z')
        angle_y = _find_angle(normal_vector,'y')
        angle_x = _find_angle(normal_vector,'x')
        
        # save the angle in an array
        normal_angle[i] = angle
        norm_ang_x[i] = angle_x
        norm_ang_y[i] = angle_y
        # save the vector in an array
        normal_vectors[i] = normal_vector
        

        # Find the eigenvalues of the matrix
        eival1, eival2, eival3 = pca.singular_values_

        # calculate the following values and assign them to the correct spot in the array
        linearity[i] = (eival1 - eival2) / eival1
        planarity[i] = (eival2 - eival3) / eival1
        scattering[i] = eival3 / eival1
        omnivariance[i] = (eival1 * eival2 * eival3) ** (1 / 3)
        anisotropy[i] = (eival1 - eival3) / eival1
        eigenentropy[i] = -1 * (
            eival1 * np.log(eival1) + eival2 * np.log(eival2) + eival3 * np.log(eival3)
        )
        eigsum[i] = eival1 + eival2 + eival3
        ch_cur[i] = eival3 / (eival1 + eival2 + eival3)

    pca_geo_array = np.vstack(
        [
            normal_angle,
            norm_ang_x,
            norm_ang_y,
            linearity,
            planarity,
            scattering,
            omnivariance,
            anisotropy,
            eigenentropy,
            eigsum,
            ch_cur,
        ]
    ).T

    return pca_geo_array


In [ ]:
# run the geoetric featuers
# NOTE this cell is *slow* (~4minutes?)
pca_results = pca_3d_shape(all_points, tree)


In [ ]:
# assign the calculated angle to the new dimension created above
subset.normal_angle = pca_results[:,0]

# save the file
subset.write("./data/subsetted_pk_code_20dec.las")

## <a name="subsetLoad"></a>Loading subset from las file
subset already has normals and other computations

In [14]:
subset = laspy.read(r"./data/subsetted_pk_code_20dec.las")
# subset = laspy.read(r"./data/fme_classifir.las")

In [15]:
for name in subset.point_format.dimension_names:
    # need to handle different methods for views that return arrays and views that return subviews
    try:
        mean_ = subset[name].mean()
    except:
        mean_ = "N/A"

    print(f"|{name}|{subset[name].max()}|{subset[name].min()}|{mean_}|  |")

|X|506423|336425|429292.5475783807|  |
|Y|447831|347833|394849.7615190117|  |
|Z|45237|8403|19564.184924570316|  |
|intensity|3402|64|942.8189832865345|  |
|return_number|5|1|N/A|  |
|number_of_returns|5|1|N/A|  |
|synthetic|0|0|N/A|  |
|key_point|0|0|N/A|  |
|withheld|0|0|N/A|  |
|overlap|0|0|N/A|  |
|scanner_channel|0|0|N/A|  |
|scan_direction_flag|1|0|N/A|  |
|edge_of_flight_line|1|0|N/A|  |
|classification|0|0|0.0|  |
|user_data|7|6|6.481019619766423|  |
|scan_angle|-3000|-4667|-3794.8262143866364|  |
|point_source_id|103|102|102.55177239640155|  |
|gps_time|270040552.1816593|270039062.8002339|270039884.3703773|  |
|red|65280|11776|34933.534884552784|  |
|green|65280|12544|34257.16123262926|  |
|blue|65280|9984|31645.35646450736|  |
|nir|0|0|0.0|  |
|Deviation|65535.0|0.0|603.9157977572299|  |
|Reflectance|19.290000915527344|-14.789999961853027|-5.444933312188677|  |
|Amplitude|34.02000045776367|0.6399999856948853|9.42818983316471|  |
|normal_angle|89.99967179015843|0.0157254648448

In [16]:
df_data_dict={}
for i in subset.point_format.dimension_names:
    if type(subset[i])==laspy.point.dims.SubFieldView:
        df_data_dict[i] = np.array(subset[i])
    else:
        df_data_dict[i] = subset[i]


las_df = pd.DataFrame(data=df_data_dict)

las_df.loc[:,'x'] = np.array(subset.x, dtype=np.float64)
las_df.loc[:,'y'] = np.array(subset.y, dtype=np.float64)
las_df.loc[:,'z'] = np.array(subset.z, dtype=np.float64)

# las_df.loc[:,'XYZ_point'] = [Point(xyz) for xyz in list(zip( las_df.X, las_df.Y, las_df.Z  ))]
las_df.loc[:,'xy_point']  = [Point(xyz) for xyz in list(zip( las_df.x, las_df.y))]

las_df.head()

,X,Y,Z,intensity,return_number,number_of_returns,synthetic,key_point,withheld,overlap,scanner_channel,scan_direction_flag,edge_of_flight_line,classification,user_data,scan_angle,point_source_id,gps_time,red,green,blue,nir,Deviation,Reflectance,Amplitude,normal_angle,x,y,z,xy_point
0,336463,400324,10399,1098,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,42752,40448,36096,0,4.0,-3.81,10.98,3.072439,89316.463,472880.324,10.399,POINT (89316.463 472880.324)
1,336575,400651,10389,1141,1,1,0,0,0,0,0,0,1,0,6,-4333,102,2.700391e+08,41216,38912,34560,0,5.0,-3.38,11.41,0.504383,89316.575,472880.651,10.389,POINT (89316.575 472880.651)
2,336432,399149,10422,1058,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,54272,48896,44544,0,7.0,-4.21,10.58,3.330684,89316.432,472879.149,10.422,POINT (89316.432 472879.149)
3,336544,399481,10422,1046,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,39168,33792,29440,0,2.0,-4.33,10.46,1.291285,89316.544,472879.481,10.422,POINT (89316.54399999999 472879.481)
4,336648,399791,10426,1171,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,40448,35072,30720,0,0.0,-3.08,11.71,6.358025,89316.648,472879.791,10.426,POINT (89316.648 472879.791)


## Loading the labelled shapefile
labels to be attached to the points by spatial join<br>
Labels are:
| Class     | Value |
|-----------|-------|
| Trees     | 1     |
| Grass     | 2     |
| Buildings | 3     |
| Sand      | 4     |
| Road      | 5     |

In [17]:
# fh = open()

classDF = gpd.read_file('./data/labels_classes1.shp', epsg=28992)
classDF

,label,geometry
0,1,"POLYGON ((89486.424 472827.832, 89450.065 4728..."
1,1,"POLYGON ((89427.050 472851.667, 89429.490 4728..."
2,2,"POLYGON ((89387.859 472927.832, 89392.457 4729..."
3,2,"POLYGON ((89386.965 472900.832, 89391.052 4728..."
4,2,"POLYGON ((89428.994 472845.580, 89432.513 4728..."
5,2,"POLYGON ((89316.424 472827.832, 89316.424 4728..."
6,2,"POLYGON ((89343.036 472899.087, 89343.674 4729..."
7,2,"POLYGON ((89361.595 472895.299, 89361.680 4729..."
8,2,"POLYGON ((89368.406 472927.832, 89382.453 4729..."
9,4,"POLYGON ((89321.560 472927.832, 89340.205 4729..."


In [18]:
gdf = gpd.GeoDataFrame(las_df, crs="EPSG:28992", geometry=las_df.xy_point)
gdf


,X,Y,Z,intensity,return_number,number_of_returns,synthetic,key_point,withheld,overlap,scanner_channel,scan_direction_flag,edge_of_flight_line,classification,user_data,scan_angle,point_source_id,gps_time,red,green,blue,nir,Deviation,Reflectance,Amplitude,normal_angle,x,y,z,xy_point,geometry
0,336463,400324,10399,1098,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,42752,40448,36096,0,4.0,-3.81,10.98,3.072439,89316.463,472880.324,10.399,POINT (89316.463 472880.324),POINT (89316.463 472880.324)
1,336575,400651,10389,1141,1,1,0,0,0,0,0,0,1,0,6,-4333,102,2.700391e+08,41216,38912,34560,0,5.0,-3.38,11.41,0.504383,89316.575,472880.651,10.389,POINT (89316.575 472880.651),POINT (89316.575 472880.651)
2,336432,399149,10422,1058,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,54272,48896,44544,0,7.0,-4.21,10.58,3.330684,89316.432,472879.149,10.422,POINT (89316.432 472879.149),POINT (89316.432 472879.149)
3,336544,399481,10422,1046,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,39168,33792,29440,0,2.0,-4.33,10.46,1.291285,89316.544,472879.481,10.422,POINT (89316.54399999999 472879.481),POINT (89316.544 472879.481)
4,336648,399791,10426,1171,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,40448,35072,30720,0,0.0,-3.08,11.71,6.358025,89316.648,472879.791,10.426,POINT (89316.648 472879.791),POINT (89316.648 472879.791)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894603,336715,347967,15341,1273,1,1,0,0,0,0,0,1,0,0,7,-3833,103,2.700406e+08,38656,35584,32000,0,10.0,-1.92,12.73,11.995350,89316.715,472827.967,15.341,POINT (89316.715 472827.967),POINT (89316.715 472827.967)
894604,336651,348302,15447,1190,1,1,0,0,0,0,0,1,0,0,7,-3833,103,2.700406e+08,33792,30720,27136,0,9.0,-2.75,11.90,20.143103,89316.651,472828.302,15.447,POINT (89316.651 472828.302),POINT (89316.651 472828.302)
894605,336584,348641,15538,1184,1,1,0,0,0,0,0,1,0,0,7,-3833,103,2.700406e+08,38144,35072,31488,0,9.0,-2.81,11.84,14.914380,89316.584,472828.641,15.538,POINT (89316.584 472828.641),POINT (89316.584 472828.641)
894606,336514,348966,15551,1186,1,1,0,0,0,0,0,1,0,0,7,-3833,103,2.700406e+08,40192,37120,33536,0,24.0,-2.80,11.86,13.877220,89316.514,472828.966,15.551,POINT (89316.514 472828.966),POINT (89316.514 472828.966)


In [19]:
# load the classes shapefile and attach class information
def class_returner(pt_xy, class_df=classDF):
    for ind in classDF.index:
        if pt_xy.intersects(classDF.loc[ind, 'geometry'])  == True: #implying there is intersection
            
            return classDF.loc[ind, 'label']
        else:
            return np.nan


In [20]:
joined = gdf.sjoin(classDF,how='inner')
joined

,X,Y,Z,intensity,return_number,number_of_returns,synthetic,key_point,withheld,overlap,scanner_channel,scan_direction_flag,edge_of_flight_line,classification,user_data,scan_angle,point_source_id,gps_time,red,green,blue,nir,Deviation,Reflectance,Amplitude,normal_angle,x,y,z,xy_point,geometry,index_right,label
157,336462,390478,10471,1113,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,59392,55808,53504,0,2.0,-3.69,11.13,4.318320,89316.462,472870.478,10.471,POINT (89316.462 472870.478),POINT (89316.462 472870.478),10,4
193,336498,389197,10432,1119,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,31744,27392,22784,0,0.0,-3.62,11.19,1.873462,89316.498,472869.197,10.432,POINT (89316.49800000001 472869.197),POINT (89316.498 472869.197),10,4
194,336613,389523,10397,1100,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,47872,44288,40960,0,6.0,-3.81,11.00,5.965852,89316.613,472869.523,10.397,POINT (89316.613 472869.523),POINT (89316.613 472869.523),10,4
195,336706,389805,10413,1082,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,58368,54784,51968,0,1.0,-3.99,10.82,6.392302,89316.706,472869.805,10.413,POINT (89316.70600000001 472869.805),POINT (89316.706 472869.805),10,4
196,336818,390143,10437,1156,1,1,0,0,0,0,0,0,0,0,6,-4333,102,2.700391e+08,57600,54016,51200,0,4.0,-3.25,11.56,5.251820,89316.818,472870.143,10.437,POINT (89316.818 472870.143),POINT (89316.818 472870.143),10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872996,342174,447003,11347,1248,1,1,0,0,0,0,0,1,0,0,7,-3167,103,2.700406e+08,49408,46080,41984,0,2.0,-2.40,12.48,14.044496,89322.174,472927.003,11.347,POINT (89322.174 472927.003),POINT (89322.174 472927.003),9,4
872997,342114,447257,11363,1210,1,1,0,0,0,0,0,1,0,0,7,-3167,103,2.700406e+08,48640,45312,41216,0,5.0,-2.78,12.10,11.323908,89322.114,472927.257,11.363,POINT (89322.114 472927.257),POINT (89322.114 472927.257),9,4
872998,342042,447567,11388,1250,1,1,0,0,0,0,0,1,0,0,7,-3167,103,2.700406e+08,48896,45568,41472,0,2.0,-2.38,12.50,16.741890,89322.042,472927.567,11.388,POINT (89322.042 472927.567),POINT (89322.042 472927.567),9,4
873355,341840,447385,11325,1223,1,1,0,0,0,0,0,1,0,0,7,-3167,103,2.700406e+08,51200,47872,43776,0,4.0,-2.65,12.23,9.023677,89321.840,472927.385,11.325,POINT (89321.84 472927.385),POINT (89321.840 472927.385),9,4


In [22]:
print( np.unique(joined.label) )
joined.to_pickle('./data/classfied_df.pkl')

[1 2 3 4 5]


# Random Forest
## test train split

In [25]:
# load from pkl file, the classified df
labelled_df = pd.read_pickle('./data/classfied_df.pkl')
labelled_df.drop(['x','y','z','xy_point', 'index_right',  'edge_of_flight_line', 'classification', 'user_data', 'point_source_id', 'gps_time', 'nir'], inplace=True, axis=1)
labelled_df.rename(columns={"label_right": "class"}, inplace=True)
labelled_df = gpd.GeoDataFrame(labelled_df, crs="EPSG:28992", geometry=labelled_df.geometry)

labelled_df.head()

,X,Y,Z,intensity,return_number,number_of_returns,synthetic,key_point,withheld,overlap,scanner_channel,scan_direction_flag,scan_angle,red,green,blue,Deviation,Reflectance,Amplitude,normal_angle,geometry,label
157,336462,390478,10471,1113,1,1,0,0,0,0,0,0,-4333,59392,55808,53504,2.0,-3.69,11.13,4.318320,POINT (89316.462 472870.478),4
193,336498,389197,10432,1119,1,1,0,0,0,0,0,0,-4333,31744,27392,22784,0.0,-3.62,11.19,1.873462,POINT (89316.498 472869.197),4
194,336613,389523,10397,1100,1,1,0,0,0,0,0,0,-4333,47872,44288,40960,6.0,-3.81,11.00,5.965852,POINT (89316.613 472869.523),4
195,336706,389805,10413,1082,1,1,0,0,0,0,0,0,-4333,58368,54784,51968,1.0,-3.99,10.82,6.392302,POINT (89316.706 472869.805),4
196,336818,390143,10437,1156,1,1,0,0,0,0,0,0,-4333,57600,54016,51200,4.0,-3.25,11.56,5.251820,POINT (89316.818 472870.143),4


In [28]:
X = labelled_df.loc[:, 'intensity':'normal_angle']  
y =labelled_df['label']

# uses a 75 25 split ratio
X_scaled = StandardScaler().fit_transform(X)
X_train, x_test, Y_train, y_test = train_test_split(X_scaled, y, random_state=10)

In [30]:
Y_train
y_test

479443    2
522335    2
69717     3
687253    1
617282    1
         ..
204263    1
531984    5
733093    3
547112    5
703619    1
Name: label, Length: 123804, dtype: int64

In [34]:
classifier_dic = {
          'RandomForestClassifier_10': RandomForestClassifier(n_estimators=10),
          'RandomForestClassifier_50': RandomForestClassifier(n_estimators=50),
          'RandomForestClassifier_100': RandomForestClassifier(n_estimators=100)
          }


def classification_func(classifier_dic, X_train, Y_train, x_test, y_test, X, y):   

    for name,classifier in classifier_dic.items():
        
        print(f'/////////////////// heyyyy we are now classifying using \t\t {name} ')
        
        classifier.fit(X_train, Y_train)
        pred = classifier.predict(x_test)
        score = classifier.score(x_test, y_test)
        print(pred)
        print(score)
        # scores = cross_val_score(classifier, X, y , scoring='accuracy', cv=7) #X needs scaling
        # print(f"Mean score: {scores.mean()}, Std dev: {scores.std()}")


        # print(f"R2 train:{r2_score(Y_train, classifier.predict(X_train))}")
        # print(f"R2 test :{r2_score(y_test,pred)}")
        # # print(f"MSE:     {mean_squared_error(y_test,pred)}")
        # # print(f"MAE:     {mean_absolute_error(y_test, pred)}")
        # print(f"MAPE:    {mean_absolute_percentage_error(y_test, pred)}")

In [35]:
classification_func(classifier_dic,  X_train, Y_train, x_test, y_test, X_scaled, y)

/////////////////// heyyyy we are now classifying using 		 RandomForestClassifier_10 
[2 2 3 ... 3 5 1]
0.9485638590029402
/////////////////// heyyyy we are now classifying using 		 RandomForestClassifier_50 
[2 2 3 ... 3 5 1]
0.9541937255662176
/////////////////// heyyyy we are now classifying using 		 RandomForestClassifier_100 
[2 2 3 ... 3 5 1]
0.9547914445413719


### workflow:

1. split the dataset into train and test
1. use BAG for buildings, sand manually, tree manually and grass manually, road manually using predefined shapes (preferably made in QGIS, saved as geojson file)
1. use the signatures for labelling the training dataset
1. train random forest on training, use a dictionary with different param values as done for biomass estimation at space4good
1. test classification prediction

### TODO
* fix df
* make current bbox into a geojson, push to ./data
* make training shapes
* think of more params to make for classification
* import code from random forest prediction of biomass for S4G

In [ ]:
subset.scan_direction_flag

## Checking that the angles with the vertical makes sense

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection="3d")
min_max_scaler = MinMaxScaler()
color = min_max_scaler.fit_transform(
    np.vstack([subset.red, subset.green, subset.blue]).T
)

p = ax.scatter(subset.X, subset.Y, subset.Z, s=1, c=subset.normal_angle)
ax.set_zlim3d(bottom=subset.Z.min())

plt.colorbar(p, shrink=0.75, label="Angle with the vertical axis")
plt.show()


In [ ]:
## TODO delete this cell later when subset is finalized
# create a markdown table with max and min, to copy in to the cell below

for name in subset.point_format.dimension_names:
    # need to handle different methods for views that return arrays and views that return subviews
    try:
        mean_ = subset[name].mean()
    except:
        mean_ = "N/A"

    print(f"|{name}|{subset[name].max()}|{subset[name].min()}|{mean_}|  |")



a) Investigate the data set: describe each of its columns and assess the spread in values in each column. What is the meaning of each attribute?

| Column name         | max                | min                  | average            | description |
| ------------------- | ------------------ | -------------------- | ------------------ | ----------- |
| X                   | 469999             | 370001               | 419845.4310215496  |             |
| Y                   | 437999             | 338001               | 387008.1474969034  |             |
| Z                   | 38855              | 12340                | 21268.85938497757  |             |
| intensity           | 3402               | 64                   | 972.2795605443956  |             |
| return_number       | 5                  | 1                    | N/A                |             |
| number_of_returns   | 5                  | 1                    | N/A                |             |
| synthetic           | 0                  | 0                    | N/A                |             |
| key_point           | 0                  | 0                    | N/A                |             |
| withheld            | 0                  | 0                    | N/A                |             |
| overlap             | 0                  | 0                    | N/A                |             |
| scanner_channel     | 0                  | 0                    | N/A                |             |
| scan_direction_flag | 1                  | 0                    | N/A                |             |
| edge_of_flight_line | 1                  | 0                    | N/A                |             |
| classification      | 0                  | 0                    | 0.0                |             |
| user_data           | 7                  | 6                    | 6.450293893628971  |             |
| scan_angle          | -3167              | -4667                | -3831.702651966856 |             |
| point_source_id     | 103                | 102                  | 102.53481523938483 |             |
| gps_time            | 270040551.7498966  | 270039063.185381     | 270039859.1351014  |             |
| red                 | 65280              | 13568                | 37609.126151147495 |             |
| green               | 65280              | 14848                | 36093.92951169035  |             |
| blue                | 65280              | 12288                | 33891.48290108402  |             |
| nir                 | 0                  | 0                    | 0.0                |             |
| Deviation           | 65535.0            | 0.0                  | 667.5175162910932  |             |
| Reflectance         | 19.290000915527344 | -14.789999961853027  | -5.168461232916284 |             |
| Amplitude           | 34.02000045776367  | 0.6399999856948853   | 9.722795606371532  |             |
| normal_angle        | 89.99985593635134  | 0.001076368705341224 | 42.61863884341944  |             |

## Spread in RBG values

In [ ]:
plt.hist(subset.green, bins=20, color="green", histtype="step")
plt.hist(subset.red, bins=20, color="red", histtype="step")
plt.hist(subset.blue, bins=20, color="blue", histtype="step")
plt.show()


b) What are ways to create a subset? Select a suitable subset of the data and visualize it in 3D. Start small, and if computer and software permits, try a bit larger subset


c) Identify at least four different object classes. Choose your classes such that together these cover a large majority of your points.

1. building
2. grass
3. pavement
4. tree
5. beach

d) Analyse which spatial scales and spectral properties are useful to distinguish your classes.

The following properties are typical of each of the classes

1. building
   1. taller than the surroundings 
2. grass
   1. high NDVI
   2. 
3. pavement
   1. 
4. tree
   1. taller that it is wide or long
5. beach

e) Extract training data for each of your object classes. Divide your training data in two parts, one for training, and one for validation. What could be the influence of  imbalances in your training data? How could your division in training and validation data affect your results?


f) Find a suitable implementation of the Random Forest algorithm. What are its parameters? What would be good settings for these parameters, given your classification task? Apply Random Forest on your data using only your observed features, to make sure your setup is correct.


# Questions
## Describe the properties of the data (sub)set that you will classify: 
### How many points? 

In [ ]:
print(f'The selected subset contains {len(subset)} points')

### What area does it cover? 

In [ ]:
print(
    f"Bounding box used to create the subset is between {xmin} and {xmax} longitude and {ymin} and {ymax} latitude (referenced to the coordinates in the LAZ file [EPSG:28992])"
)


### What observed features will you use? 
### Visualize your final subset, including the useful observed features. 

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection="3d")
min_max_scaler = MinMaxScaler()
color = min_max_scaler.fit_transform(
    np.vstack([subset.red, subset.green, subset.blue]).T
)


print(color.shape)
ax.scatter(subset.X, subset.Y, subset.Z, s=1, c=color)
ax.set_zlim3d(bottom=subset.Z.min())

plt.show()



### Describe your at least four different object classes; 

For each point determine neighborhoods of k1, k2, ... points (using e.g. an efficient data structure), and use these k1, k2, ... points to estimate several feature values.



**2. Describe $\gt$ 20 different geometric attributes, obtained using at least 2 different neighborhood sizes, to characterize your points. What are the dimensions of the data covariance matrix you use to compute the geometric features? Give one example on how you determine the PCA eigenvalues of one k-neighborhood. Indicate for each feature how it could help to distinguish your classes, given also the neighborhood sizes you consider.**

The geometric attributes were computed using a covariance matrix in 3D. To describe the geometry of real world features, fitting the PCA plane in 3D makes the most sense. For every point in the subset, the nearest *k* points were found using a KDtree, which is an efficient data structure for finding the nearest features in 3D space. When working with such a large cloud, the computational expense is very important to be practical to run on a personal computer. Once the k nearest neighbors are found, a PCA model is trained on the XYZ location of the k-neighborhood, which returns 3 vectors (the eigenvectors of the Covariance matrix). The three vectors point in direction of the most variance, the 2nd most variance, and normal to the other two, respectively. The trained PCA model also returns the eigenvalues covariance matrix, which can be used to compute the following features:

3. Compute all geometric features for all points in your subset using Python. Visualize selected results, e.g. by combining features in a false color visualization and/or using histograms. Which features are best at discriminating your classes? Why?

In [ ]:
def fc_vis_and_hist(point_cloud_subset,geometric_array,name):
    # create a 3d visualization
    fig = plt.figure(figsize=(24, 10))
    ax = fig.add_subplot(1,2,1, projection="3d")

    assert len(point_cloud_subset) == len(geometric_array)

    p = ax.scatter(subset.X, subset.Y, subset.Z, s=1, c=geometric_array)
    plt.colorbar(p,shrink=0.75,label=name)
    ax.set_zlim3d(bottom=subset.Z.min())
    # plt.title(name)

    # plt.show()
    
    # create a histogram
    # plt.figure(figsize=(12,7))
    ax = fig.add_subplot(1,2,2)

    ax.hist(geometric_array,bins=20)
    plt.suptitle(name)
    plt.xlabel(f'values of {name}')
    plt.ylabel('Count of points')
    plt.show()

In [ ]:
geometric_feat_names = [
            'Normal Angle',
            'Angle to X axis',
            'Angle to Y axis',
            'Linearity',
            'Planarity',
            'Scattering',
            'Omnivariance',
            'Anisotropy',
            'Eigen Entropy',
            'Eigen Sum',
            'Change in Curvature',
        ]


for column,name in zip(pca_results.T,geometric_feat_names):
    fc_vis_and_hist(subset,column,name)

4. Describe and visualize your training data. Is your training data balanced? Make sure that a zone of your point cloud data is really ’unseen’, that is that no training data is taken from that zone, so you can inspect if Random Forest also works there.


5. Feed your training data to Random Forest using at least 10 of your best geometric and observed features. What settings did you use?


6. Classify your point cloud data and visualize and discuss your result. What went well? Give also examples where the classifier mixed up classes. What are possible explanations for these confusions? How are the classification results on the unseen zone?